# Celda inicial
Monta la carpeta de google drive donde se encuentra todo el codigo y los datos.

Tambien pullea el repositorio de git, para tener la ultima version del codigo

In [2]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)
# MODIFICAR CON PATH DONDE SE ENCUENTRA LA CARPETA EN EL DRIVE
import os
path = '/content/gdrive/MyDrive/Pasantia/DEEPscreen/DEEPScreen'
os.chdir(path)

# actualiza el repositorio
#!git pull

# actualiza las dependencias
!pip -q install -r ./requirements.txt

import pandas as pd
import numpy as np

Mounted at /content/gdrive/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 KB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


# Importando la df de entenamiento 

In [ ]:
df = pd.read_csv('./data_procesing_filter/df_train_mini.csv', index_col = 0, header = 0)
df = df[df.columns.drop('CHEMBL1293224')]
df

In [ ]:

df = pd.read_csv('./bin/trial/df_trial.csv', index_col = 0, header = 0)
df = df.rename(columns={'CHEMBL286':'CHEMBL286_AUTORES'})
df = df.reset_index()
df

In [ ]:
df = pd.read_csv('./data_procesing_filter/df_mini_mejores_mas_amplio.csv', index_col = 0, header = 0)
df = df[df.columns.drop('CHEMBL1293224')]
df

,comp_id,CHEMBL1293311,CHEMBL1806,CHEMBL2169736,CHEMBL2361,CHEMBL2443,CHEMBL2967,CHEMBL3085613,CHEMBL3238,CHEMBL4161,CHEMBL4506,CHEMBL4803,CHEMBL5131,CHEMBL5445,CHEMBL5847,smiles
0,CHEMBL10009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CCCC(=O)NCCc1c2n(c3ccc(OC)cc13)Cc1ccccc1-2
4,CHEMBL100473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N#CC(=C(/N)Sc1ccccc1N)/C(C#N)=C(/N)Sc1ccccc1N
7,CHEMBL100716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,CCCN(CCC)c1nc(N)nc2c1NC(c1ccc(OC)cc1)CN2
9,CHEMBL1008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC(C)COCC(CN(Cc1ccccc1)c1ccccc1)N1CCCC1
13,CHEMBL101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CCCCC1C(=O)N(c2ccccc2)N(c2ccccc2)C1=O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53534,CHEMBL98547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,Nc1nc(NCC2CCCCC2)c2c(n1)NCC(c1ccc(Cl)cc1)N2
53536,CHEMBL986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC1=C(/C=C/C(C)=C/C=C/C(C)=C/CO)C(C)(C)CCC1
53541,CHEMBL99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC(/C=C/C(=O)NO)=C\\[C@@H](C)C(=O)c1ccc(N(C)C)cc1
53565,CHEMBL99636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,CN(CCC[C@H](N)C(=O)O)C(=N)N


# Entrenamiento

In [ ]:
import sys
sys.path.insert(0, "bin")
from deepscreen_trypanosomatics import train
from logging_deepscreen import logger
import pandas as pd

In [ ]:
class deepscreen_db:
    def __init__(self,db_path:str, logger):
        self.logger = logger
        import sqlite3
        try:
            self.con = sqlite3.connect(db_path)
        except Exception as exp:
            self.logger.error(f'Unable to conect to deepscreen db\n{exp}')
        self.cur = self.con.cursor()
        self._check_create_table()
    
    def get_trained_models(self):
        trained_models = self.cur.execute('''
        SELECT target_id FROM trained_models
        ''').fetchall()
        trained_models_plain_list = [tup[0] for tup in trained_models]
        self.logger.debug(f'Trained models queried from db {trained_models_plain_list[:3]}... total trained = {len(trained_models)}')
  
        return trained_models_plain_list
    
    def add_trained_model(self,target_id,trained_model_matrix_path,test_values_dict:dict):
        data = [target_id,
                trained_model_matrix_path, 
                test_values_dict['Precision'],
                test_values_dict['Recall'],
                test_values_dict['F1-Score'],
                test_values_dict['Accuracy'],
                test_values_dict['MCC'],
                int(test_values_dict['TP']),
                int(test_values_dict['FP']),
                int(test_values_dict['TN']),
                int(test_values_dict['FN']),
                ]
        self.logger.debug(f'Trained model results to be stored in db: {test_values_dict}')
        try:
            self.cur.execute('''
            INSERT INTO trained_models (target_id,
                trained_model_matrix_path, 
                precision,
                recall,
                f1_score,
                accuracy,
                mcc,
                true_positive,
                false_positive,
                true_negative,
                false_negative)
            VALUES (?,?,?,?,?,?,?,?,?,?,?);
            ''', data)
            self.con.commit()
            return True
        except Exception as exp:
            self.logger.error(f'Unable to write trained model in db\nFollowing exeption raised{exp}')
            return False

    def _check_create_table(self):
        tables_raw = self.cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
        if len(tables_raw) > 0:
            tables = tables_raw[0]
        
            if not 'trained_models' in tables:
                self.logger.debug('trained_models table unexistent')
                self.cur.execute('''
                CREATE TABLE trained_models (
                    target_id VARCHAR(255),
                    trained_model_matrix MEDIUMBLOB,
                    trained_model_matrix_path MEDIUMTEXT,
                    precision FLOAT(255,10),
                    recall FLOAT(255,10),
                    f1_score FLOAT(255,10),
                    accuracy FLOAT(255,10),
                    mcc FLOAT(255,10),
                    true_positive INT(255),
                    false_positive INT(255),
                    true_negative INT(255),
                    false_negative INT(255)
                );
                '''
                )
                tables_raw = self.cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
                tables = tables_raw[0]
                self.logger.debug('trained_models table created')
                self.logger.debug(f'tables in db: {tables}')

        else:
            self.logger.debug('trained_models table unexistent')
            self.cur.execute('''
            CREATE TABLE trained_models (
                target_id VARCHAR(255),
                trained_model_matrix MEDIUMBLOB,
                trained_model_matrix_path MEDIUMTEXT,
                precision FLOAT(255,10),
                recall FLOAT(255,10),
                f1_score FLOAT(255,10),
                accuracy FLOAT(255,10),
                mcc FLOAT(255,10),
                true_positive INT(255),
                false_positive INT(255),
                true_negative INT(255),
                false_negative INT(255)
            );
            '''
            )
            tables_raw = self.cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
            tables = tables_raw[0]
            self.logger.debug('trained_models table created')
            self.logger.debug(f'tables in db: {tables}')

class trainer:
    def __init__(self, df, db_path:str, logger):
        self.logger = logger


        self.df = self._check_correct_df(df)

        self._config_nn = {
            'fully_layer_1':512,
            'fully_layer_2':256,
            'learning_rate':0.001,
            'batch_size':32,
            'drop_rate':0.1,
            'n_epoch':200
        }
        self.db = deepscreen_db(db_path,logger)
    
    def get_config_nn(self):
        return self._config_nn
    
    def change_config_nn(self, full_config:dict=None, fully_layer_1:int=None,  fully_layer_2:int=None, learning_rate:float=None, batch_size:int=None, drop_rate:float=None, n_epoch:int=None):
        
        if full_config != None:
            self._config_nn = full_config
            logger.debug(f'Neural Network config parameters changed to {self._config_nn}')
        
        if fully_layer_1 != None:
            self._config_nn['fully_layer_1'] = fully_layer_1
            logger.debug(f'Neural Network config parameters changed to {self._config_nn}')

        if fully_layer_2 != None:
            self._config_nn['fully_layer_2'] = fully_layer_2
            logger.debug(f'Neural Network config parameters changed to {self._config_nn}')
        
        if learning_rate != None:
            self._config_nn['learning_rate'] = learning_rate
            logger.debug(f'Neural Network config parameters changed to {self._config_nn}')
            
        if batch_size != None:
            self._config_nn['batch_size'] = batch_size
            logger.debug(f'Neural Network config parameters changed to {self._config_nn}')
            
        if drop_rate != None:
            self._config_nn['drop_rate'] = drop_rate
            logger.debug(f'Neural Network config parameters changed to {self._config_nn}')
            
        if n_epoch != None:
            self._config_nn['n_epoch'] = n_epoch
            logger.debug(f'Neural Network config parameters changed to {self._config_nn}')

    def train(self, result_path:str, tmp_imgs:bool=False):

        targets = self._get_target_list(self.df)

        for target in targets:
            if target in self.db.get_trained_models():
                logger.info(f'{target} target skipped because it was allready processed')
                continue

            if tmp_imgs:
                import tempfile
                with tempfile.TemporaryDirectory(prefix=f'{target}_') as tmpdirname:
                    logger.debug(f'training {target}')
                    logger.debug(f'tmp images mode on. imgs temporaly stored in {tmpdirname}')
                    images = tmpdirname
                    config_nn = self.get_config_nn()
                    df_training = self.df[['comp_id',target,'smiles']]
                    df_training = df_training.dropna(how='any')
                    training_matrix_path, test_values = train(df_training,target,result_path,images,experiment_name=target,train_split_mode='train_random_split',model_name='CNNModel1',**config_nn)
                    self.logger.debug(f'Matrix stored in {training_matrix_path}; Results values {test_values}')
                    self.db.add_trained_model(target,training_matrix_path,test_values)

            else:    
                logger.debug(f'training {target}')
                images = result_path+f'/imgs_{target}/'
                logger.debug(f'molecules imgs stored in {images}')
                config_nn = self.get_config_nn()
                df_training = self.df[['comp_id',target,'smiles']]
                df_training = df_training.dropna(how='any')
                training_matrix_path, test_values = train(df_training,target,result_path,images,experiment_name=target,train_split_mode='train_random_split',model_name='CNNModel1',**config_nn)
                self.logger.debug(f'Matrix stored in {training_matrix_path}; Results values {test_values}')
                self.db.add_trained_model(target,training_matrix_path,test_values)
        
        self.logger.debug(f'Training of {targets} succeded')
        return True

    def _get_target_list(self, df):
        targets_to_train = df.columns.to_list()
        try:
            targets_to_train.remove('comp_id')
            targets_to_train.remove('smiles')
        except:
            self.logger.error('comp_id and smiles column not found')
        self.logger.debug(f'targets for training: {targets_to_train[:3]}...(total {len(targets_to_train)})')
        return targets_to_train

    def _check_correct_df(self,df):
        df = df.copy()

        df_columns = df.columns

        if not (('comp_id' in df_columns) or ('smiles' in df_columns)):
            error = 'Issues with the df. "comp_id" or "smiles" column missing'
            self.logger.error(error)
            raise RuntimeError(error)
        
        df_columns_targets = df_columns.drop(['comp_id','smiles'])

        if len(df_columns_targets) < 1:
            error = 'Issues with the df. Target columns Missing'
            self.logger.error(error)
            raise RuntimeError(error)

        dtypes = df[df_columns_targets].dtypes
        any_not_int64 = (dtypes != 'Int64').any()
        if any_not_int64:
            self.logger.warning('There are some column with dtype diferent to "Int64. This issue is gonna get solved with convert_dtypes')
            df = df.convert_dtypes(convert_integer=True)
        
        return df
    


In [ ]:
trainer = trainer(df,'./deepscreen.db', logger)

2023-02-10 20:20:57:WARNING: There are some column with dtype diferent to "Int64. This issue is gonna get solved with convert_dtypes


In [ ]:
trainer.train('./results', tmp_imgs=True)

2023-02-10 20:21:02:DEBUG: targets for training: ['CHEMBL1293311', 'CHEMBL1806', 'CHEMBL2169736']...(total 14)
2023-02-10 20:21:02:DEBUG: Trained models queried from db ['CHEMBL1293311', 'CHEMBL1806', 'CHEMBL2169736']... total trained = 13
2023-02-10 20:21:02:INFO: CHEMBL1293311 target skipped because it was allready processed
2023-02-10 20:21:02:DEBUG: Trained models queried from db ['CHEMBL1293311', 'CHEMBL1806', 'CHEMBL2169736']... total trained = 13
2023-02-10 20:21:02:INFO: CHEMBL1806 target skipped because it was allready processed
2023-02-10 20:21:02:DEBUG: Trained models queried from db ['CHEMBL1293311', 'CHEMBL1806', 'CHEMBL2169736']... total trained = 13
2023-02-10 20:21:02:INFO: CHEMBL2169736 target skipped because it was allready processed
2023-02-10 20:21:02:DEBUG: Trained models queried from db ['CHEMBL1293311', 'CHEMBL1806', 'CHEMBL2169736']... total trained = 13
2023-02-10 20:21:02:INFO: CHEMBL2361 target skipped because it was allready processed
2023-02-10 20:21:02:DEB

True

In [11]:
!git commit -m 'update to colab'

^C
